# Part I. ETL Pipeline for Pre-Processing the Files

#### ***Import Python packages***

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### ***Creating list of filepaths to process original event csv data files***

In [2]:
# checking your current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### ***Processing the files to create the data file csv that will be used for Apache Casssandra tables***

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            

print(len(full_data_rows_list))


# creating a smaller event data csv file called event_datafile_new csv that is used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# checking the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Creating tables on Apache Cassandra 

## The CSV file titled <font color=red>event_datafile_new.csv</font> is used as a source. This file contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### ***Creating a Cluster***

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### ***Create Keyspace***

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)
    
except Exception as e:
    print(e)

#### ***Set Keyspace***

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

In [8]:
#It's used to create tables on Cassandra. 
def create_table(query,session):
    try:
        session.execute(query)
        print('table is created.')
    except Exception as e:
        print(e)

In [9]:
# It's used to insert related records into related tables.
def insert_into_table(file,query,func,session):
    ## Read the CSV file and insert records into table.        
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            session.execute(query, func(line) )
    

### **You may find the results of three different cases below**

### ***Case 1: The artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4***

In [10]:
## Create table 'songs'. 
## PRIMARY KEY: is a composite key which is combination of session_id and item_in_session.
query = """CREATE TABLE IF NOT EXISTS songs 
        (
         session_id int, 
         item_in_session int, 
         artist text, 
         length float,
         song text, 
         PRIMARY KEY (session_id, item_in_session)
         )"""

create_table(query,session)                 

table is created.


In [11]:
#It returns related items for songs table.
def get_songs_record(line):
    return (int(line[8]), int(line[3]), line[0], float(line[5]), line[9])
    
# Read the CSV file and insert data into 'songs' table. 
file = 'event_datafile_new.csv'

query= """ INSERT INTO songs(session_id, item_in_session, artist, length, song) 
             VALUES (%s, %s, %s, %s, %s)"""

insert_into_table(file, query, get_songs_record, session)



#### ***Run the code below to see the results***

In [12]:
query = "select artist, length, song from songs where session_id = 338 and item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

row_arr=[ row for row in rows ]

df= pd.DataFrame(row_arr, columns=['artist','length','song'])

df.head()



,artist,length,song
0,Faithless,495.307312,Music Matters (Mark Knight Dub)


### ***Case 2: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182***

In [13]:
## Create table 'user_activity' 
## PRIMARY KEY:is a compound key consisting of a composite partition key (user_id,session_id) 
#  and a clustering key (item_in_session). 
#  item_in_session column is choosed as clustering key because based on the question songs should be sorted by this column.

query = """CREATE TABLE IF NOT EXISTS user_activity (
        user_id int,
        session_id int, 
        item_in_session int, 
        artist text, 
        first_name text,
        last_name text,
        song text, 
        PRIMARY KEY ( (user_id,session_id), item_in_session) )""" 

create_table(query,session)    
 

table is created.


In [14]:
#It returns related items for user_activity table.
def get_user_activty_record(line):
    return (int(line[10]), int(line[8]), int(line[3]), line[0], line[1], line[4], line[9] )

file = 'event_datafile_new.csv'

query= """INSERT INTO user_activity(user_id, session_id, item_in_session, artist, first_name, last_name, song)
            VALUES (%s, %s, %s, %s, %s, %s, %s)"""

insert_into_table(file, query, get_user_activty_record, session)

 #### ***Run the code below to see the results***

In [15]:
query = "select artist, song, first_name, last_name from user_activity where user_id=10 and session_id=182 "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
row_arr=[ row for row in rows ]

df= pd.DataFrame(row_arr, columns=['artist','song','first_name','last_name' ])

df.head()



,artist,song,first_name,last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


### ***Case 3:All user names (first and last) in the music app history who listened to the song 'All Hands Against His Own'***

In [16]:
## Create table 'user_list' 
## PRIMARY KEY: is a composite key consisting of song and user_id.  

query = """CREATE TABLE IF NOT EXISTS user_list (
            song text,
            user_id int,
            first_name text,
            last_name text,            
            PRIMARY KEY (song, user_id )
            )"""

create_table(query, session)

    

table is created.


In [17]:
#It returns related items for user_list table.
def get_user_list_record(line):
    return (line[9],int(line[10]), line[1], line[4] )
    
query= """ INSERT INTO user_list(song, user_id, first_name, last_name) 
             VALUES (%s, %s, %s, %s )"""

insert_into_table(file, query, get_user_list_record, session)



#### ***Run the code below to see the results***

In [18]:
query = "select first_name, last_name from user_list where song='All Hands Against His Own' "
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

row_arr=[ row for row in rows ]

df= pd.DataFrame(row_arr, columns=['first_name','last_name' ])

df.head()


,first_name,last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


 #### ***Drop the tables before closing out the sessions***

In [19]:
def drop_table(table_name):
    query="""DROP TABLE IF EXISTS """+table_name
    try:
        session.execute(query)
        print('dropped: {}'.format(table_name))
    except Exception as e:
        print(e)
        
drop_table('songs')
drop_table('user_activity')
drop_table('user_list')

dropped: songs
dropped: user_activity
dropped: user_list


### ***Close the session and cluster connection***

In [20]:
session.shutdown()
cluster.shutdown()